In [1]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine, event
import pandas.io.sql as psql
import numpy as np
import datetime

print('Выполнено:',datetime.datetime.now())

Выполнено: 2021-07-15 19:33:37.478680


In [3]:
#список пользователей по leader-id  и email из ple
fb=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/fb.csv', \
                              sep=';',encoding='ANSI')
fb.head()

,contextID,track_id,track,theme,eventUUID,eventTitle,event_id,a_title,startDate,leaderID,answer_hash,feedbackQuestionID,question,os_id,value,createDT,guid
0,287,305,Общий трек,Встреча с командой организаторов,d5c82259-e78a-444e-85cd-95f1a2b5b064,Вводная лекция для организаторов интенсива,51821,Вводная лекция для организаторов интенсива,2021-03-02 07:00:00,795262,1616105927-48149,44,"Насколько вероятно, что Вы порекомендуете меро...",385654,10,2021-03-18 22:18:47,p1_ss21_student_free
1,287,305,Общий трек,Встреча с командой организаторов,d5c82259-e78a-444e-85cd-95f1a2b5b064,Вводная лекция для организаторов интенсива,51821,Вводная лекция для организаторов интенсива,2021-03-02 07:00:00,795262,1616105927-48149,46,"Перечислите основные этапы деятельности, в кот...",385655,NaN,2021-03-18 22:18:47,p1_ss21_student_free
2,287,305,Общий трек,Встреча с командой организаторов,d5c82259-e78a-444e-85cd-95f1a2b5b064,Вводная лекция для организаторов интенсива,51821,Вводная лекция для организаторов интенсива,2021-03-02 07:00:00,795262,1616105927-48149,43,"Чему новому, как вам кажется, вы научились?",385656,NaN,2021-03-18 22:18:47,p1_ss21_student_free
3,287,305,Общий трек,Встреча с командой организаторов,d5c82259-e78a-444e-85cd-95f1a2b5b064,Вводная лекция для организаторов интенсива,51821,Вводная лекция для организаторов интенсива,2021-03-02 07:00:00,795262,1616105927-48149,55,Была ли в этом ценность и в чем она выражается?,385657,NaN,2021-03-18 22:18:47,p1_ss21_student_free
4,287,305,Общий трек,Встреча с командой организаторов,d5c82259-e78a-444e-85cd-95f1a2b5b064,Вводная лекция для организаторов интенсива,51821,Вводная лекция для организаторов интенсива,2021-03-02 07:00:00,795262,1616105927-48149,45,Поделитесь вашими комментариями или пожеланиям...,385658,NaN,2021-03-18 22:18:47,p1_ss21_student_free


In [4]:
fb_pivot = pd.pivot_table(fb,index=['leaderID','event_id','createDT'], columns='feedbackQuestionID', values='value', \
                               aggfunc = 'max'). \
                reset_index().fillna(0)
fb_pivot.head()

feedbackQuestionID,leaderID,event_id,createDT,43,44,45,46,55,93,94,...,230,231,232,233,234,235,236,237,238,239
0,56159,51991,2021-04-02 08:50:25,0,0,0,0,0,0,0,...,359e674d-96eb-49c0-9afb-a519fafeb9c7,0,7,8,1,9,0,"1. У вас интересная инженерная задача, но не п...",0,8
1,56159,51991,2021-04-02 09:45:04,0,0,0,0,0,0,0,...,ced634c0-633a-4eda-8de7-9ed59c98d295,0,6,5,2,9,0,1. Вернитесь к формулировке проблемы и решения...,0,7
2,56159,51991,2021-04-02 10:33:40,0,0,0,0,0,0,0,...,bf9170fb-f893-46f2-b376-2660727c1676,0,6,5,5,6,0,1. Определиться с пользователем будущего решен...,0,7
3,56159,51991,2021-04-02 10:40:46,0,0,0,0,0,0,0,...,0632fe5b-9988-4bbe-bddc-8be99ebddbc5,0,7,5,2,7,0,"1. Отличный паспорт проекта, видно, что продел...",0,5
4,114107,51992,2021-04-03 08:55:39,0,0,0,0,0,0,0,...,0a848af4-9045-4a0b-9b01-c1b53103dd94,0,6,6,6,6,0,0,0,10


In [32]:
fb[fb.feedbackQuestionID == 44][['question','feedbackQuestionID']].head(1)

,question,feedbackQuestionID
0,"Насколько вероятно, что Вы порекомендуете меро...",44


In [56]:
dict_1 = fb.groupby(['feedbackQuestionID']).agg(question_text=('question', 'max')).fillna(0).reset_index()
dict_1 = dict_1.set_index('feedbackQuestionID')
dict_1.head(3)

,question_text
feedbackQuestionID,
43,"Чему новому, как вам кажется, вы научились?"
44,"Насколько вероятно, что Вы порекомендуете меро..."
45,Поделитесь вашими комментариями или пожеланиям...


In [50]:
dict_1.to_dict()

{'question_text': {43: 'Чему новому, как вам кажется, вы научились?',
  44: 'Насколько вероятно, что Вы порекомендуете мероприятие знакомым? Пожалуйста, оцените по шкале от 0 до 10',
  45: 'Поделитесь вашими комментариями или пожеланиями по поводу того, с чем вы познакомились?',
  46: 'Перечислите основные этапы деятельности, в которую вы были вовлечены и благодаря этому приобретали новые или развивали существующие компетенции',
  55: 'Была ли в этом ценность и в чем она выражается?',
  93: 'Насколько вероятно, что Вы порекомендуете мероприятие знакомым? Пожалуйста, оцените по шкале от 0 до 10',
  94: 'Что Вам понравилось в мероприятии?',
  95: 'Что на мероприятии было ценно и полезно лично для вас?',
  96: 'А что бы Вы посоветовали улучшить?',
  97: 'Ваши комментарии и замечания по поводу мероприятия и его организации.',
  195: 'Какому проекту вы сейчас выставляете оценки?',
  196: 'Если вы не нашли проект в выпадающем списке, напишите его название здесь:',
  197: 'Насколько ясно пока

In [52]:
#Переименуем в понятные всем имена - чтобы записи на мероприятия с Зумами не путались
fb_out = fb_pivot.rename(columns={ 
  43: 'Чему новому, как вам кажется, вы научились?',
  44: 'Насколько вероятно, что Вы порекомендуете мероприятие знакомым? Пожалуйста, оцените по шкале от 0 до 10',
  45: 'Поделитесь вашими комментариями или пожеланиями по поводу того, с чем вы познакомились?',
  46: 'Перечислите основные этапы деятельности, в которую вы были вовлечены и благодаря этому приобретали новые или развивали существующие компетенции',
  55: 'Была ли в этом ценность и в чем она выражается?',
  93: 'Насколько вероятно, что Вы порекомендуете мероприятие знакомым? Пожалуйста, оцените по шкале от 0 до 10',
  94: 'Что Вам понравилось в мероприятии?',
  95: 'Что на мероприятии было ценно и полезно лично для вас?',
  96: 'А что бы Вы посоветовали улучшить?',
  97: 'Ваши комментарии и замечания по поводу мероприятия и его организации.',
  195: 'Какому проекту вы сейчас выставляете оценки?',
  196: 'Если вы не нашли проект в выпадающем списке, напишите его название здесь:',
  197: 'Насколько ясно показаны пользователи и проблема?',
  198: 'Насколько проработан прототип у команды?',
  199: 'Насколько предлагаемое решение соответствует проблеме?',
  200: 'Насколько поставленная цель адекватна силам и компетенциям команды?',
  201: 'Качество презентации',
  202: 'Какие рекомендации по доработке прототипа у вас есть?',
  203: 'Комментарии по дальнейшему развитию проекта в целом:',
  218: 'Какому проекту вы сейчас выставляете оценки?',
  219: 'Если вы не нашли проект в выпадающем списке, напишите его название здесь:',
  220: 'Насколько ясно показаны пользователи и проблема?',
  221: 'Насколько проработан прототип у команды?',
  222: 'Насколько предлагаемое решение соответствует проблеме?',
  223: 'Насколько поставленная цель адекватна силам и компетенциям команды?',
  224: 'Качество презентации',
  225: 'Какие рекомендации по доработке проекта у вас есть? Порекомендуйте команде полезные ресурсы (например, акселерационные или образовательные программы)',
  226: 'Оставьте свои данные, если вы хотите связаться с командой',
  228: 'Можете ли вы сказать, что мероприятие было хорошо структурировано?',
  229: 'Насколько ясно показаны пользователи и проблема?',
  230: 'Выберите проект, которому вы выставляете оценку:',
  231: 'Если вы не нашли проект в выпадающем списке, напишите его название здесь:',
  232: 'Насколько предлагаемое решение соответствует проблеме?',
  233: 'Насколько хорошо выполнен анализ аналогов?',
  234: 'Общение с пользователями / заказчиком, информативность промежуточных результатов исследования:',
  235: 'Насколько хорошо команда понимает, что именно и как она будет делать в рамках интенсива. Насколько реалистичен масштаб проекта, а поставленная цель адекватна силам и компетенциям команды?',
  236: 'Технологический стек.',
  237: 'Ваши комментарии и рекомендации команде. (Будут переданы участникам команды, отвечайте по желанию)',
  238: 'Комментарии по команде для организаторов и наставника команды. (Не будут переданы участникам команды, отвечайте по желанию)',
  239: 'Оцените, как прошла сессия, насколько открытой для обратной связи и вовлеченной в процесс обсуждения была команда?'
                        })
fb_out.head(3)

feedbackQuestionID,leaderID,event_id,createDT,"Чему новому, как вам кажется, вы научились?","Насколько вероятно, что Вы порекомендуете мероприятие знакомым? Пожалуйста, оцените по шкале от 0 до 10","Поделитесь вашими комментариями или пожеланиями по поводу того, с чем вы познакомились?","Перечислите основные этапы деятельности, в которую вы были вовлечены и благодаря этому приобретали новые или развивали существующие компетенции",Была ли в этом ценность и в чем она выражается?,"Насколько вероятно, что Вы порекомендуете мероприятие знакомым? Пожалуйста, оцените по шкале от 0 до 10",Что Вам понравилось в мероприятии?,...,"Выберите проект, которому вы выставляете оценку:","Если вы не нашли проект в выпадающем списке, напишите его название здесь:",Насколько предлагаемое решение соответствует проблеме?,Насколько хорошо выполнен анализ аналогов?,"Общение с пользователями / заказчиком, информативность промежуточных результатов исследования:","Насколько хорошо команда понимает, что именно и как она будет делать в рамках интенсива. Насколько реалистичен масштаб проекта, а поставленная цель адекватна силам и компетенциям команды?",Технологический стек.,"Ваши комментарии и рекомендации команде. (Будут переданы участникам команды, отвечайте по желанию)","Комментарии по команде для организаторов и наставника команды. (Не будут переданы участникам команды, отвечайте по желанию)","Оцените, как прошла сессия, насколько открытой для обратной связи и вовлеченной в процесс обсуждения была команда?"
0,56159,51991,2021-04-02 08:50:25,0,0,0,0,0,0,0,...,359e674d-96eb-49c0-9afb-a519fafeb9c7,0,7,8,1,9,0,"1. У вас интересная инженерная задача, но не п...",0,8
1,56159,51991,2021-04-02 09:45:04,0,0,0,0,0,0,0,...,ced634c0-633a-4eda-8de7-9ed59c98d295,0,6,5,2,9,0,1. Вернитесь к формулировке проблемы и решения...,0,7
2,56159,51991,2021-04-02 10:33:40,0,0,0,0,0,0,0,...,bf9170fb-f893-46f2-b376-2660727c1676,0,6,5,5,6,0,1. Определиться с пользователем будущего решен...,0,7


In [54]:
with pd.ExcelWriter('C:/Users/d.muravlyansky/Documents/UNTI/feedback_pivot.xlsx') as writer:  
    fb_out.to_excel(writer, sheet_name='Feedbsck',index=False)
    dict_1.reset_index().to_excel(writer, sheet_name='Типы вопросов',index=False)